# Qiskit to OpenQASM3 Superconducting and IonTrap

Three steps:

0. Choose a Quantum Algorithm. Using Berstein Vazirani for a 3 bit hidden string s.
1. Creating transpiled circuit at optimization_level=3 with Superconducting backend to OpenQASM3. 
2. Creating transpiled circuit at optimization_level=3 with Ion Trap backend to OpenQASM3. 


## Step 0: Write the Bernstein Vazirani algorithm for a 3 bit hidden string s, in Qiskit.


In [1]:
# Write simplified Bernstein Vazirani algorithm from scratch using Qiskit
from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer
from qiskit.compiler import transpile


s = '011'   # the hidden binary string
n = 3 # number of bits used to represent s


# We need a circuit with n qubits, plus one auxiliary qubit
# We also need n classical bits to write the output to
qiskit_untranspiled_qc = QuantumCircuit(n+1, n)

# Put auxiliary in the minus state |->
# We leave the other n qubits in just |0>
qiskit_untranspiled_qc.h(n)
qiskit_untranspiled_qc.z(n)

# Apply Hadamard gates to each of the n "main" qubits (excluding the auxiliary)
# before querying the oracle
for i in range(n):
    qiskit_untranspiled_qc.h(i)
    
# Apply barrier
qiskit_untranspiled_qc.barrier()

# Apply the inner-product oracle
s = s[::-1] # reverse s to fit qiskit's qubit ordering
for q in range(n):
    if s[q] == '0':
        pass #qiskit_untranspiled_qc.i(q)
    else:
        qiskit_untranspiled_qc.cx(q, n)
        
# Apply barrier
qiskit_untranspiled_qc.barrier()

# Apply Hadamard gates after querying the oracle
for i in range(n):
    qiskit_untranspiled_qc.h(i)


# Measurement
for i in range(n):
    qiskit_untranspiled_qc.measure(i, i)


qiskit_untranspiled_qc = transpile(qiskit_untranspiled_qc, backend=None)  # No Optimization and no mapping because we have not specified a backend.
print('No Optimization')
print('Depth:', qiskit_untranspiled_qc.depth())
print('Gate counts:', qiskit_untranspiled_qc.count_ops())
qiskit_untranspiled_qc.draw("mpl").savefig("Images/B-V_qiskit_untranspiled_qc.jpg")

No Optimization
Depth: 6
Gate counts: OrderedDict([('h', 7), ('measure', 3), ('barrier', 2), ('cx', 2), ('z', 1)])


#### Logger
In order to track and report information about transpilation, we'll use a Logger object to help us gain insight into the transpiler.

In [2]:
import logging

logging.basicConfig(level='DEBUG')
logging.getLogger('qiskit.transpiler').setLevel('INFO')
formatter = logging.Formatter('%(asctime)s --- %(name)s --- %(levelname)s: %(message)s\n')
handler = logging.getLogger().handlers[0]
handler.setFormatter(formatter)

## Step 1: Qiskit to OpenQASM3 using IBM Superconducting Qubits
1. Load IBM Superconducting Backend. 
2. Transpilation Optimization_Level=3 with Superconducting Backend. Save DAG, and histogram results.
3. Convert transpiled (with IBM Superconducting Backend) circuit to OpenQasm code.

#### Substep 1: Load IBM Superconducting Backend. 

In [3]:
# Define a simulated noisy backend
from qiskit.test.mock import FakeBoeblingen
from qiskit.compiler import transpile

IBM_Superconducting_backend = FakeBoeblingen()


2022-07-11 06:21:52,606 --- qiskit.compiler.assembler --- INFO: Total Assembly Time - 0.07820 (ms)

2022-07-11 06:21:52,617 --- qiskit.compiler.assembler --- INFO: Total Assembly Time - 0.05317 (ms)

2022-07-11 06:21:52,618 --- qiskit.compiler.assembler --- INFO: Total Assembly Time - 0.09298 (ms)

2022-07-11 06:21:52,619 --- qiskit.compiler.assembler --- INFO: Total Assembly Time - 0.07701 (ms)

2022-07-11 06:21:52,620 --- qiskit.compiler.assembler --- INFO: Total Assembly Time - 0.04077 (ms)

2022-07-11 06:21:52,620 --- qiskit.compiler.assembler --- INFO: Total Assembly Time - 0.03195 (ms)

2022-07-11 06:21:52,621 --- qiskit.compiler.assembler --- INFO: Total Assembly Time - 0.03004 (ms)

2022-07-11 06:21:52,621 --- qiskit.compiler.assembler --- INFO: Total Assembly Time - 0.10085 (ms)

2022-07-11 06:21:52,622 --- qiskit.compiler.assembler --- INFO: Total Assembly Time - 0.03934 (ms)

2022-07-11 06:21:52,622 --- qiskit.compiler.assembler --- INFO: Total Assembly Time - 0.09990 (ms)



#### Substep 2: Transpilation Optimization_Level=3 with Superconducting Backend. Save DAG, and histogram results.

In [4]:
transpiled_Superconducting_qc3 = transpile(qiskit_untranspiled_qc, IBM_Superconducting_backend, optimization_level=3)  # MAXIMUM Optimization!!!
print('Optimization Level 3')
print('Depth:', transpiled_Superconducting_qc3.depth())
print('Gate counts:', transpiled_Superconducting_qc3.count_ops())
transpiled_Superconducting_qc3.draw("mpl").savefig("Images/B-V_transpiled_Superconducting_qc3.jpg")

2022-07-11 06:21:53,744 --- qiskit.quantum_info.synthesis.two_qubit_decompose --- DEBUG: Requested fidelity: None calculated fidelity: 1.0 actual fidelity 1.0

2022-07-11 06:21:53,746 --- qiskit.transpiler.runningpassmanager --- INFO: Pass: UnitarySynthesis - 0.02027 (ms)

2022-07-11 06:21:53,746 --- qiskit.transpiler.runningpassmanager --- INFO: Pass: Unroll3qOrMore - 0.00882 (ms)

2022-07-11 06:21:53,746 --- qiskit.transpiler.runningpassmanager --- INFO: Pass: RemoveResetInZeroState - 0.00906 (ms)

2022-07-11 06:21:53,746 --- qiskit.transpiler.runningpassmanager --- INFO: Pass: OptimizeSwapBeforeMeasure - 0.00715 (ms)

2022-07-11 06:21:53,747 --- qiskit.transpiler.runningpassmanager --- INFO: Pass: RemoveDiagonalGatesBeforeMeasure - 0.07820 (ms)

2022-07-11 06:21:53,747 --- qiskit.transpiler.runningpassmanager --- INFO: Pass: SetLayout - 0.00381 (ms)

2022-07-11 06:21:53,751 --- qiskit.transpiler.runningpassmanager --- INFO: Pass: VF2Layout - 4.15611 (ms)

2022-07-11 06:21:53,752 ---

Optimization Level 3
Depth: 5
Gate counts: OrderedDict([('u2', 7), ('measure', 3), ('barrier', 2), ('cx', 2)])


2022-07-11 06:21:54,006 --- matplotlib.font_manager --- DEBUG: findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=8.0.

2022-07-11 06:21:54,007 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/Users/hakanayaz/opt/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/STIXSizTwoSymReg.ttf', name='STIXSizeTwoSym', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,008 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/Users/hakanayaz/opt/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans-Oblique.ttf', name='DejaVu Sans', style='oblique', variant='normal', weight=400, stretch='normal', size='scalable')) = 1.05

2022-07-11 06:21:54,008 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/Users/hakanayaz/opt/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/STIXNonUni

2022-07-11 06:21:54,018 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/Users/hakanayaz/opt/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf', name='DejaVu Sans', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 0.05

2022-07-11 06:21:54,019 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/Users/hakanayaz/opt/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/STIXNonUniBolIta.ttf', name='STIXNonUnicode', style='italic', variant='normal', weight=700, stretch='normal', size='scalable')) = 11.335

2022-07-11 06:21:54,019 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/Users/hakanayaz/opt/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/STIXGeneralBolIta.ttf', name='STIXGeneral', style='italic', variant='normal', weight=700, stretch='normal', size='scalable')) = 11.335

2022-07-11 06:21:54,019 --- matplotlib.font_

2022-07-11 06:21:54,029 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/Galvji.ttc', name='Galvji', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,030 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/NotoSansPhoenician-Regular.ttf', name='Noto Sans Phoenician', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,030 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Thonburi.ttc', name='Thonburi', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,031 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/ヒラギノ角ゴシック W5.ttc', name='Hiragino Sans', style='normal', variant='normal', weight=500, stretch='normal', 

...

2022-07-11 06:21:54,105 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/NotoSansTaiViet-Regular.ttf', name='Noto Sans Tai Viet', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,105 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/SFNSMono.ttf', name='.SF NS Mono', style='normal', variant='normal', weight=295, stretch='normal', size='scalable')) = 10.14975

2022-07-11 06:21:54,106 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/AppleGothic.ttf', name='AppleGothic', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,106 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/SFNSItalic.ttf', name='System Font', style='italic', variant='normal', weight=400, stretch='norm

2022-07-11 06:21:54,116 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/Apple Chancery.ttf', name='Apple Chancery', style='normal', variant='normal', weight=0, stretch='normal', size='scalable')) = 10.43

2022-07-11 06:21:54,116 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/Luminari.ttf', name='Luminari', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,117 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/Tahoma.ttf', name='Tahoma', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,117 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/Al Tarikh.ttc', name='Al Tarikh', style='normal', variant='normal', weight=400, stretch='normal', siz

2022-07-11 06:21:54,127 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/HelveticaNeue.ttc', name='Helvetica Neue', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,128 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/Kailasa.ttc', name='Kailasa', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,128 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/Georgia Italic.ttf', name='Georgia', style='italic', variant='normal', weight=400, stretch='normal', size='scalable')) = 11.05

2022-07-11 06:21:54,128 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/Diwan Kufi.ttc', name='Diwan Kufi', style='normal', variant='normal', weight=400, stretch='normal', size='

2022-07-11 06:21:54,137 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Symbol.ttf', name='Symbol', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,138 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/NotoSansHatran-Regular.ttf', name='Noto Sans Hatran', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,138 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/KufiStandardGK.ttc', name='KufiStandardGK', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,139 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/STIXSizFiveSymReg.otf', name='STIXSizeFiveSym', style='normal', variant='normal', weight=4

2022-07-11 06:21:54,148 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/ArialHB.ttc', name='Arial Hebrew', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,149 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/NotoSansNewTaiLue-Regular.ttf', name='Noto Sans New Tai Lue', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,149 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/ITFDevanagari.ttc', name='ITF Devanagari', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05

2022-07-11 06:21:54,149 --- matplotlib.font_manager --- DEBUG: findfont: score(FontEntry(fname='/System/Library/Fonts/Supplemental/Farisi.ttf', name='Farisi', style='normal', variant='normal', weight=400, st

In [5]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.dagcircuit import DAGCircuit
from qiskit.converters import circuit_to_dag
from qiskit.tools.visualization import dag_drawer

# Required software to visually see DAG
!pip install pydot

# Draw DAG of the transpiled circuit that used optimization_level=3
dag = circuit_to_dag(transpiled_Superconducting_qc3)
# dag_drawer(dag, filename="Images/B-V_transpiled_Superconducting_qc3_dag.jpg")

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [ ]:
from qiskit.tools.visualization import plot_histogram

results_BV_Lev3 = IBM_Superconducting_backend.run(transpiled_Superconducting_qc3).result()
counts = results_BV_Lev3.get_counts()
plot_histogram(counts, filename="Images/B-V_transpiled_Superconducting_qc3_histogram.jpg")

2022-07-11 06:21:55,731 --- qiskit.compiler.assembler --- INFO: Total Assembly Time - 0.09608 (ms)



#### Substep3: Convert transpiled (with IBM Superconducting Backend) circuit to OpenQasm code.

In [ ]:
# Qiskit Circuit  <-->  .qasm file
from qiskit import qasm3

# Qiskit -> OpenQASM3
# Print
print(qasm3.dumps(transpiled_Superconducting_qc3))
# Dump to file
with open("B-V_transpiled_Superconducting_qc3.qasm", 'w+') as fp:
    qasm3.dump(transpiled_Superconducting_qc3, fp)

## Step 2: Qiskit to OpenQASM3 using AQT IonTrap Qubits
1. Load the AQT Ion Trap Backend 
2. Transpilation Optimization_Level=3 with IonTrap Backend. Save DAG, and histogram results.
3. Convert transpiled (with AQT IonTrap Backend) circuit to OpenQasm code.

#### Substep 1: Load the AQT Ion Trap Backend
1. To use the AQT Ion Trap backend in Qiskit you must first install the following in your terminal: `pip install qiskit-aqt-provider`

2. To gain access to the AQT Ion Trap backend you must make an AQT cloud access account **[HERE](https://www.aqt.eu/qc-systems/)**, and use the API key provided by AQT.

For more information about using the AQT Ion Trap backend in Qiskit, please click **[HERE](https://qiskit.org/documentation/partners/aqt/install.html).**


In [ ]:
from qiskit_aqt_provider import AQTProvider

# Input AQT Cloud access API key.
aqt = AQTProvider('151d4eeaea5e4e438dca82276ecf524e')
print("List of AQT Ion Trap Backends: ", aqt.backends(), "\n")

# Choose an AQT Backend. 
AQT_IonTrap_backend = aqt.get_backend('aqt_qasm_simulator_noise_1')

In [ ]:
from qiskit.tools.jupyter import *
from qiskit.transpiler import CouplingMap
import pprint
pp = pprint.PrettyPrinter(indent=4)


# Print out basic features of this backend. Note, I am using a function found in Backend_General_Information
print("---------------------------------Backend Configuration---------------------------------")

config = AQT_IonTrap_backend.configuration()
pp.pprint(config.to_dict())
#cm = CouplingMap(config.coupling_map)

print("---------------------------------Backend Properties---------------------------------")
# Print out information about the Properties of the backend.
#properties = AQT_IonTrap_backend.properties()
#pp.pprint(properties.to_dict())

#cm.draw()

#### (Optional Substep): Optimization_Level=0

In [ ]:
transpiled_IonTrap_qc0 = transpile(qiskit_untranspiled_qc, AQT_IonTrap_backend, optimization_level=0)
transpiled_IonTrap_qc0.draw("mpl").savefig("Images/B-V_transpiled_IonTrap_qc0.jpg")

In [ ]:
# Draw DAG of the transpiled circuit that used optimization_level=0
dag = circuit_to_dag(transpiled_IonTrap_qc0)
dag_drawer(dag, filename="Images/B-V_transpiled_IonTrap_qc0_dag.jpg")

In [ ]:
# Lets execute the transpiled circuit on the simulated noisy backend
results = AQT_IonTrap_backend.run(transpiled_IonTrap_qc0).result()

# Lets get the results of the executed circuit
counts = results.get_counts()
plot_histogram(counts, filename="Images/B-V_transpiled_IonTrap_qc0_histogram.jpg")

#### Substep 2: Transpilation Optimization_Level=3 with IonTrap Backend.

In [ ]:
transpiled_IonTrap_qc3 = transpile(qiskit_untranspiled_qc, AQT_IonTrap_backend, optimization_level=3)
transpiled_IonTrap_qc3.draw('mpl').savefig("Images/B-V_transpiled_IonTrap_qc3.jpg")

In [ ]:
# Draw DAG of the transpiled circuit that used optimization_level=3
dag = circuit_to_dag(transpiled_IonTrap_qc3)
dag_drawer(dag, filename="Images/B-V_transpiled_IonTrap_qc3_dag.jpg")

In [ ]:
# Lets execute the transpiled circuit on the simulated noisy backend
results = AQT_IonTrap_backend.run(transpiled_IonTrap_qc3).result()

# Lets get the results of the executed circuit
counts = results.get_counts()
plot_histogram(counts, filename="Images/B-V_transpiled_IonTrap_qc3_histogram.jpg")

#### Substep 3: Convert transpiled (with AQT IonTrap Backend) circuit to OpenQasm code.

In [ ]:
# Qiskit Circuit  <-->  .qasm file
from qiskit import qasm3

# Qiskit -> OpenQASM3
# Print
print(qasm3.dumps(transpiled_IonTrap_qc3))
# Dump to file
with open("B-V_transpiled_IonTrap_qc3.qasm", 'w+') as fp:
    qasm3.dump(transpiled_IonTrap_qc3, fp)